### Alternatives to the Markowitz Portfolio Problem

Solved via QP
- Minimum Variance Objective
- Mean Variance Objective
- Mean Variance Objective with Biased Estimators

Solved via Split Bregman
- Mean Variance Objective with Elastic Net Penalty
- Mean Variance Objective with Biased Estimators and Elastic Net Penalty

In [2]:
import sys
import numpy as np
path='/home/jbohn/jupyter/personal/L1_Portfolio_Opt/'
sys.path.append(path)

from Features.features import load_features
from Split_Bregman.split_bregman_opt import split_bregman
from Parameter_Estimation.bias_cov import reg_cov
from Parameter_Estimation.bias_mean import reg_mean


### Prep Data and Features

In [4]:
train_test_split=0.5
train_data, test_data = load_features(train_test_split)
train_data

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,-0.006283,-0.012860,-0.008112,0.000000,-0.024321,0.006161,0.001768,0.005387,0.016313,-0.005385,...,-0.002244,0.000411,0.059041,-0.011930,0.000000,-0.003426,-0.001745,0.031165,0.034651,0.000000
2010-01-06,0.014082,0.000454,0.005538,0.000000,-0.015358,0.010575,-0.005314,-0.002532,-0.002124,-0.002350,...,0.015971,0.003796,-0.013204,0.001919,0.000000,-0.007175,-0.007717,-0.000323,0.083382,0.000000
2010-01-07,0.000717,0.002946,0.008250,0.000000,0.000000,-0.000935,-0.024271,-0.010511,-0.019595,-0.000471,...,-0.002952,0.009377,0.021131,-0.004322,0.000000,-0.000288,-0.025318,0.022681,0.106160,0.000000
2010-01-08,0.007022,0.014602,0.005099,0.000000,-0.020446,-0.003986,-0.008223,-0.012567,-0.005436,-0.001414,...,-0.001109,0.011700,-0.007191,0.000481,0.000000,0.000288,-0.003256,-0.021228,-0.016320,0.000000
2010-01-11,-0.004040,-0.004694,0.005073,0.000000,-0.018394,-0.000940,-0.000918,0.000648,-0.013169,0.004469,...,0.009939,0.004203,-0.003245,0.009574,0.000000,0.017133,0.003256,0.021860,0.006075,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-24,0.000539,0.003030,-0.000438,0.028495,0.003386,-0.001094,0.005914,-0.001129,0.004828,0.000822,...,-0.000883,-0.003186,-0.005222,0.022329,0.001032,0.001512,0.003197,0.005624,-0.000351,0.020044
2014-12-26,-0.004202,0.007623,0.003277,0.011563,0.004669,-0.000657,0.003434,0.005820,-0.000937,-0.000469,...,-0.000442,0.000428,-0.003518,0.009614,0.004890,0.004797,-0.002941,0.000526,0.001402,0.008178
2014-12-29,0.002703,0.000883,-0.005467,0.002386,-0.004149,-0.008142,-0.002943,-0.005443,-0.007258,-0.003881,...,-0.003320,0.006861,0.017339,0.018150,-0.002313,0.005726,-0.000640,-0.005886,0.005587,-0.004308


### Mean-Variance

In [ ]:
cum_returns_split_i=np.cumprod(1+np.dot(w2,returns_test.T))
cum_returns_split_j=np.cumprod(1+np.dot(wi,returns_test.T))
cum_returns_split_opt=np.cumprod(1+np.dot(w_opt,returns_test.T))


equal=(1/returns_test.shape[1])*np.ones(returns_test.shape[1])
cum_returns_equal=np.cumprod(1+np.dot(equal,returns_test.T))
fig,ax=plt.subplots(figsize=(10,8))
plt.title("Out of Sample Performance")
plt.plot(pd.to_datetime(returns_test.index),cum_returns_split_i,label='Split Bregamn Sparse',color='red')
plt.plot(pd.to_datetime(returns_test.index),cum_returns_split_j,label='Split Bregamn Optimal',color='green')
plt.plot(pd.to_datetime(returns_test.index),cum_returns_equal,label='Equal Weighted',color='blue')
plt.plot(pd.to_datetime(returns_test.index),cum_returns_split_opt,label='Markowitz Mean Variance',color='purple')
plt.xlabel(f"{len(wi[nonzero_wi])} Nonzero components")
plt.ylabel("Cumulative Return")
plt.legend()

In [ ]:
def get_sharpe(port_returns,rf=0.0178):
    sd=np.std(port_returns)*np.sqrt(252)
    mu=np.mean(port_returns)*252
    
    sharpe=(mu-rf)/sd
    
    return sharpe

In [ ]:
print(f"Sharpe of equal allocation is {round(get_sharpe(np.dot(equal,returns_test.T)),2)}")
print(f"Sharpe of Mean Variance allocation is {round(get_sharpe(np.dot(w_opt,returns_test.T)),2)}")
print(f"Sharpe of Split-Bregman Optimal allocation is {round(get_sharpe(np.dot(wi,returns_test.T)),2)}")
print(f"Sharpe of Split-Bregman Sparse allocation is {round(get_sharpe(np.dot(w1,returns_test.T)),2)}")